In [ ]:
import geopandas as gpd

# Charger le shapefile
gdf = gpd.read_file("input.shp")

# Sauvegarder en GeoJSON
gdf.to_file("output.geojson", driver="GeoJSON")

In [ ]:
import geopandas as gpd
from shapely.geometry import box

# 1. Supposons que ton GeoDataFrame s'appelle dept
# et est déjà en Lambert 93 (EPSG:2154)

# 2. Délimitation :
bounds = dept.total_bounds  # (xmin, ymin, xmax, ymax)
xmin, ymin, xmax, ymax = bounds

# 3. Définir taille tuile (exemple 5km x 5km)
step = 5000 # mètres

# 4. Construire les tuiles et les découper au département
tiles = []
x = xmin
while x < xmax:
    y = ymin
    while y < ymax:
        tile_geom = box(x, y, x + step, y + step)
        # Découpe la tuile au shape du département
        tile_clip = tile_geom.intersection(dept.unary_union)
        if not tile_clip.is_empty:
            tiles.append(tile_clip)
        y += step
    x += step

# 5. Reprojeter vers WGS84 (EPSG:4326)
tiles_wgs84 = [gpd.GeoSeries([t], crs="EPSG:2154").to_crs(epsg=4326).values[0] for t in tiles]

# 6. Boucle de traitement
for i, tile_geom in enumerate(tiles_wgs84):
    download_tile(tile_geom, tile_id=i)
    # Si besoin : print(tile_geom)